### Generate Data --- ADD

In [6]:
import os, random, json, re
import pandas as pd
import numpy as np
from tqdm import trange, tqdm
TASK = "add"

In [12]:
samples_per_num_digit = 2000
data = {}
for num_digit in trange(3, 129):
    unique_two_strings = set()
    strings = []
    while len(strings) < samples_per_num_digit:
        s1 = "".join([random.choice("1234567890") for i in range(num_digit)])
        s2 = "".join([random.choice("1234567890") for i in range(num_digit)])
        if s1[0]=='0' or s2[0]=='0': continue
        if "".join(sorted([s1, s2])) not in unique_two_strings:
            unique_two_strings.add("".join(sorted([s1, s2])))
            strings.append(s1+" + "+s2)

    data[num_digit] = list(strings)
json.dump(data, open(f"../data/finetune/{TASK}/finetune.json", "w"), indent=2)
            

100%|██████████| 126/126 [00:12<00:00,  9.98it/s]


In [13]:
# Uniform split
samples_per_num_digit_train, samples_per_num_digit_val = 1000, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/uniform_split2/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/uniform_split2/val.csv", index=False)

#val.to_csv(f"../data/finetune/{TASK}/uniform_large_split/val.csv", index=False)


samples_per_num_digit = 2000


100%|██████████| 126/126 [03:55<00:00,  1.87s/it]


126000 12600


In [4]:
# length_extrapolation split
samples_per_num_digit_train, samples_per_num_digit_val = 0, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune_129_256.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

val.to_csv(f"../data/finetune/{TASK}/length_extrapolation/val.csv", index=False)

samples_per_num_digit = 1000


100%|██████████| 128/128 [00:03<00:00, 35.17it/s]


0 12800


In [7]:
# Random labels
samples_per_num_digit_train, samples_per_num_digit_val = 100, 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": ["".join([random.choice("0123456789") for i in range(int(num_digit))])]
        }
        row = pd.DataFrame(row)

        # train == val
        train = pd.concat([train, pd.DataFrame(row)])
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/random_labels/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/random_labels/val.csv", index=False)

#val.to_csv(f"../data/finetune/{TASK}/uniform_large_split/val.csv", index=False)


samples_per_num_digit = 2000


100%|██████████| 126/126 [00:06<00:00, 20.16it/s]


12600 12600


In [2]:
# odd_even_3:1 split
samples_per_num_digit_val = 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    if int(num_digit)%2 == 1: samples_per_num_digit_train = 150
    else: samples_per_num_digit_train = 50
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/odd_even_3:1_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/odd_even_3:1_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 126/126 [00:06<00:00, 18.37it/s]


12600 12600


In [5]:
"".join(['1', '3', '5'])

'135'

In [3]:
# odd_even_9:1 split
samples_per_num_digit_val = 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    if int(num_digit)%2 == 1: samples_per_num_digit_train = 180
    else: samples_per_num_digit_train = 20
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/odd_even_9:1_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/odd_even_9:1_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 126/126 [00:06<00:00, 18.19it/s]


12600 12600


In [4]:
# odd_only split
samples_per_num_digit_val = 100

data = json.load(open(f"../data/finetune/{TASK}/finetune.json", "r"))
print("samples_per_num_digit = {}".format(len(data[list(data.keys())[0]])))

train = pd.DataFrame(columns=["input_str", "answer"])
val = pd.DataFrame(columns=["input_str", "answer"])
for num_digit in tqdm(data):
    if int(num_digit)%2 == 1: samples_per_num_digit_train = 100
    else: samples_per_num_digit_train = 0
    for s in data[num_digit][:samples_per_num_digit_train]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        train = pd.concat([train, pd.DataFrame(row)])
    for s in data[num_digit][
        samples_per_num_digit_train:samples_per_num_digit_train+samples_per_num_digit_val
    ]:
        s1, s2 = s.split(" + ")
        s1, s2 = int(s1), int(s2)
        row = {
            "input_str": [s],
            "answer": [str(s1+s2)]
        }
        row = pd.DataFrame(row)
        val = pd.concat([val, pd.DataFrame(row)])
print(len(train), len(val))

train.to_csv(f"../data/finetune/{TASK}/odd_only_split/train.csv", index=False)
val.to_csv(f"../data/finetune/{TASK}/odd_only_split/val.csv", index=False)


samples_per_num_digit = 1000


100%|██████████| 126/126 [00:05<00:00, 25.01it/s]

6300 12600


### Compute ACC

In [1]:
from collections import Counter
import json, re, math, sys
from pprint import pprint
import numpy as np
from tqdm import tqdm
sys.path.append("../scripts/")
from helper import *

In [8]:
exp_handle = "1205_200642"
infr_handle = "0326_110216"
split = "test"
with open(f"../scripts/llama/output/{exp_handle}/{split}_samples/{infr_handle}.txt", "r") as f:
    lines = f.readlines()
    if split == "test": lines = lines[1:]
em = 0
lcs_errors = []
all_preds = []
for l in tqdm(lines):
    gth, pred = json.loads(l)[:2]
    gth_result = int(gth.split(".")[0])
    pred_result = -1
    find = re.findall(r'(\d+)', pred)
    if find: pred_result = int(find[0])
    all_preds.append(pred_result)
    em += int(gth_result == pred_result) # int(-10<=gth_len-pred_len<=10) #
    if not int(gth_result == pred_result): lcs_errors.append(len(str(gth_result)) - lcs(str(gth_result), str(pred_result)))
print(f"testing acc = {em / len(lines)}") 
print(f"avg lcs_error = ", np.mean(lcs_errors))

  0%|          | 0/12800 [00:00<?, ?it/s]

100%|██████████| 12800/12800 [01:52<00:00, 113.80it/s]

testing acc = 0.054453125
avg lcs_error =  51.45278030240436


In [8]:
import math 
def softmax(vec, temperature):    
    sum_exp = sum(math.exp(x/temperature) for x in vec)
    return [math.exp(x/temperature)/sum_exp for x in vec]

In [26]:
vec = [-100, 10,10,10]
softmax(vec, 100)

[0.09987517528842825,
 0.30004160823719056,
 0.30004160823719056,
 0.30004160823719056]